# U.S. Patent Phrase to Phrase Matching
Help Identify Similar Phrases in U.S. Patents

В этом наборе данных представлены пары фраз (anchor  и target) и необходимо оценить, насколько они похожи по шкале от 0 (совсем не похожи) до 1 (идентичны по смыслу). Эта задача отличается от стандартной задачи семантического сходства тем, что здесь оценка сходства проводилась в контексте патента, в частности его классификации [CPC](https://en.wikipedia.org/wiki/Cooperative_Patent_Classification) , которая указывает предмет, к которому относится патент. 

In [ ]:
from fastai.imports import *

In [ ]:
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
path = (Path('../input/us-patent-phrase-to-phrase-matching') if iskaggle
    else Path.home()/'data'/'us-patent-phrase-to-phrase-matching')
path.ls()

```id``` - уникальный идентификатор для пары фраз

```anchor``` - первая фраза

```target``` - вторая фраза

```context``` - классификация CPC (версия 2021.05), которая указывает тему, в рамках которой должно быть оценено сходство

```score``` - сходство. Получено из комбинации одного или нескольких ручных экспертных оценок.

In [ ]:
df = pd.read_csv(path/'train.csv')
df

1.0 - Очень близкое совпадение. Обычно это точное совпадение, за исключением, возможно, различий в спряжении, количестве (например, единственное или множественное число) и добавлении или удалении стоп-слов (например, “the”, “and”, “or”).

0,75 - Близкий синоним, например, “мобильный телефон” против “мобильного телефона”. Сюда также входят сокращения, например "TCP" -> "протокол управления передачей".

0,5 - Синонимы, которые не имеют одинакового значения (одна и та же функция, одни и те же свойства).

0.25 - Немного похожи, например, две фразы находятся в одной и той же области высокого уровня, но не являются синонимами. Сюда же относятся и антонимы.

0.0 - Совсем разное.


In [ ]:
df.describe(include='object')

In [ ]:
eval_df = pd.read_csv(path/'test.csv')
len(eval_df)

In [ ]:
print('TARGET','\n' )
print(df.target.value_counts())
print('\n','ANCHOR','\n' )
print(df.anchor.value_counts())
print('\n','CONTEXT','\n' )
print(df.context.value_counts())

In [ ]:
df.score.hist();

In [ ]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

In [ ]:
from datasets import Dataset,DatasetDict
from transformers import AutoModelForSequenceClassification,AutoTokenizer

ds = Dataset.from_pandas(df)
ds

In [ ]:
model_nm = '../input/deberta-v3-base/deberta-v3-base'
tokz = AutoTokenizer.from_pretrained(model_nm)

Почему deberta? Показала крутые результаты на тесте SuperGLUE, учитывает не только значения слов, но и их позиции и роли, так как у нас работа с контекстом -- это важно. Лучше чем другие модели, например, RoBERTa-Large.
Часто используется в таких задачах, короче мне понравилась :)

In [ ]:
def tok_func(x): 
    return tokz(x["input"])

In [ ]:
tok_func(ds[0])

In [ ]:
inps = "anchor","target","context"
tok_ds = ds.map(tok_func, batched=True)
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [ ]:
row = tok_ds[0]
row['input']

##  Test df

In [ ]:
eval_df.describe()

In [ ]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

In [ ]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

In [ ]:
def corr_d(eval_pred): 
    return {'pearson': np.corrcoef(*eval_pred)[0][1]}

## Training

In [ ]:
from transformers import TrainingArguments,Trainer

In [ ]:
lr = 8e-5
bs = 128
epochs = 6
wd = 0.01

args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

In [ ]:
trainer.train();

In [ ]:
metrics = [o['eval_pearson'] for o in trainer.state.log_history if 'eval_pearson' in o]
metrics[-1]

In [ ]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

In [ ]:
preds = np.clip(preds, 0, 1)

In [ ]:
preds

In [ ]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds.flatten()
})

submission.to_csv('submission.csv', index=False)

## Можем попробовать поиграться с input и параметрами

In [ ]:
lr = 8e-5
bs = 128
epochs = 4
wd = 0.02

In [ ]:
def get_dds(df):
    ds = Dataset.from_pandas(df).rename_column('score', 'label')
    tok_ds = ds.map(tok_func, batched=True)
    return tok_ds.train_test_split(0.25, seed=42)

In [ ]:
def get_model():
    args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
        evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
        num_train_epochs=4, weight_decay=wd, report_to='none')
    return AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)

def get_trainer(dds, model=None):
    if model is None: model = get_model()
    return Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                   tokenizer=tokz, compute_metrics=corr_d)

In [ ]:
df['input'] = df.input.str.lower()
dds = get_dds(df)

In [ ]:
trainer = get_trainer(dds)
trainer.train()

In [ ]:
metrics = [o['eval_pearson'] for o in trainer.state.log_history if 'eval_pearson' in o]
metrics[-1]

**На Leaderboard показала результаты**

Your Best Entry!
Your submission scored 0.8019, which is not an improvement of your previous score. Keep trying!